In [113]:
%%file perfect.lp

%%% Attempts to perfectly hash keys given as item(number)

#const mod_range = 100.
#const coef_range = 100.

%%% Choose a modulus and coefficient
1 {mod(1..mod_range)} 1.
1 {coef(1..coef_range)} 1.

%%% Reject collisions
:- coef(A), mod(M), item(I1), item(I2),  I1 != I2,  A*I1\M = A*I2\M.

Overwriting perfect.lp


In [110]:
%%file minimal.lp

%%% Minimize the range of hashed values
maxhash(X) :- coef(A), mod(M), X = #max{ A*I\M:item(I) }.
minhash(X) :- coef(A), mod(M), X = #min{ A*I\M:item(I) }.
diff(X) :- maxhash(A), minhash(B), X = A-B.
#minimize{ X:diff(X) }.

Writing minimal.lp


In [132]:
%%file ordered.lp

%%% Reject if items are out of order when hashed
:- item(I1), item(I2), I1 < I2, coef(A), mod(M),  A*I1\M > A*I2\M.

Overwriting ordered.lp


In [106]:
%%file process.py

import json
import sys
import re


result = json.load(sys.stdin)
atoms = result['Call'][0]['Witnesses'][0]['Value']

items = []
modulus = 0
coef = 0

for a in atoms:
    match = re.match('item\(([0-9]*)\)', a)
    if match:
        items += [int(match.group(1))]
    
    match = re.match('mod\(([0-9]*)\)', a)
    if match:
        modulus = int(match.group(1))
        
    match = re.match('coef\(([0-9]*)\)', a)
    if match:
        coef = int(match.group(1))

        
table = [(item*coef%modulus, item) for item in items]
table.sort()

print "Hash function: %dx mod %d" %(coef,modulus)

print "%7s %7s" % ("hash", "item")
for item in table:
    print "%7d %7d" % item

Overwriting process.py


In [134]:
!clingo items.lp perfect.lp minimal.lp quiet.lp --outf=2 --quiet=1 2>/dev/null | python2 process.py

Hash function: 67x mod 36
   hash    item
      1       7
      4     100
      6       6
     10   34234
     11       5
     13      55
     16       4
     17   23123
     21       3
     22      10
     25      67
     26       2
     27       9
     29      23
     31       1
     32       8
     33     123


In [153]:
%%file gen.py
import random

seen = {}
for i in range(5):
    x = random.randint(0,1000)
    if x not in seen:
        print "item(%d)." % (x)
        seen[x] = 1
    

Overwriting gen.py


In [154]:
!python2 gen.py > gen.lp; clingo gen.lp perfect.lp minimal.lp ordered.lp quiet.lp --outf=2 --quiet=1 2>/dev/null | python2 process.py

Hash function: 55x mod 57
   hash    item
     25      16
     38      38
     44     149
     45     291
     49     631
